# **Installing Requirements**

In [ ]:
!pip install pydicom
!pip install nvsmi

# **Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Main Model**

In [ ]:
from sklearn.metrics import classification_report
#from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
#from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
#from sklearn.svm import SVC
from pydicom import dcmread
import seaborn as sn
import pandas
import numpy
import keras
import nvsmi
import os
import cv2
import time

# well-defined initial state
numpy.random.seed(20)

# Start of Timer (Data preparation phase)
ST = time.perf_counter()

# Picture Channel Detector (Work with Channels_last)
def PicCha(InMRIs):
    MRIPic = InMRIs[0]
    Channel = 0
    MatrixShapeV1 = numpy.shape(MRIPic)
    print(numpy.shape(InMRIs))
    print(MatrixShapeV1)
    if len(MatrixShapeV1) <= 2 or MatrixShapeV1[0] == 1:
        Channel = 1
        InMRIs = numpy.reshape(InMRIs, numpy.shape(InMRIs) + (1,))
    else:
        Channel = numpy.shape(MRIPic)[0]
    return {"InMRIs":InMRIs, "Channel":Channel}

# input Control
def InCo():
    InVa = 0
    while True:
        try:
            InVa = int(input("\nEnter Number: "))
            if InVa >= 0:
                return InVa
            else:
                print("\ninput is not positive integer!\n")
        except:
            print("\ninput is not integer!\n")

# Return current time (Program Time)
def ProTim():
    return time.perf_counter()

# Matrix creator with all samples
def MaAlSa(Addr):
    global StTi, TrPa
    InMRIs, InMRsOnHo = [], []
    # Training Data Path
    TrPa = sorted(os.listdir(Addr))
    # Start time for creat matrixs
    StTi = ProTim()
    for i in range(0,len(TrPa)):
        TrSP = sorted(os.listdir(Addr+'/'+TrPa[i]+'/T1-image'))
        for j in range(0,(len(TrSP))):
            MRdcm = dcmread(Addr+'/'+TrPa[i]+'/T1-image'+'/'+TrSP[j])
            for k  in range(0,len(CSVFi.Subject_ID)):
                if str(CSVFi.Subject_ID[k]) in MRdcm.PatientName:
                    OnHoRo = [0]*6
                    OnHoRo[((CSVFi.Age[k])//10)-2] = 1
                    InMRsOnHo.append(OnHoRo)
                    InMRIs.append(MRdcm.pixel_array)
                    break
    InMRIs = numpy.array(InMRIs)
    InMRsOnHo = numpy.array(InMRsOnHo)
    MRICha = PicCha(InMRIs)
    return {"InMRIs":MRICha["InMRIs"], "InMRsOnHo":InMRsOnHo, "MRICha":MRICha["Channel"]}

# Matrix creator with some samples
def MaSoSa(Addr, NumSa = 0):
    global StTi, TrPa
    InMRIs, InMRsOnHo = [], []
    NuSa = None
    # Training Data Path
    TrPa = os.listdir(Addr)
    if (NumSa == 0):
        print("\n\nEnter number of samples to use:")
        # Number of Patients Samples use to Creat input MRIs Matrix
        NuSa = InCo()
    else:
        NuSa = NumSa
    # Start time for creat matrixs
    StTi = ProTim()
    for i in range(0,len(TrPa)):
        TrSP = sorted(os.listdir(Addr+'/'+TrPa[i]+'/T1-image'))
        for j in range((len(TrSP)//2)+(-NuSa//2),(len(TrSP)//2)+(-NuSa//2)+NuSa):
            MRdcm = dcmread(Addr+'/'+TrPa[i]+'/T1-image'+'/'+TrSP[j])
            for k  in range(0,len(CSVFi.Subject_ID)):
                if str(CSVFi.Subject_ID[k]) in MRdcm.PatientName:
                    OnHoRo = [0]*6
                    OnHoRo[((CSVFi.Age[k])//10)-2] = 1
                    InMRsOnHo.append(OnHoRo)
                    InMRIs.append(MRdcm.pixel_array)
                    break
    InMRIs = numpy.array(InMRIs)
    InMRsOnHo = numpy.array(InMRsOnHo)
    MRICha = PicCha(InMRIs)
    return {"InMRIs":MRICha["InMRIs"], "InMRsOnHo":InMRsOnHo, "MRICha":MRICha["Channel"]}

def CoMaSh(CoM):
    print("\n\nConfusion matrix:\n\n     %20s"  %"2 3 4 5 6".replace(' ', '    '))
    for i in range(numpy.shape(CoMa)[0]):
        print('\n', 2+i, ' [', end = '')
        for j in range(numpy.shape(CoMa)[0]):
            if j == numpy.shape(CoMa)[0]-1:
                print("%s" %(CoMa[i][j]), end = '')
                break
            print("%-5s" %(CoMa[i][j]), end = '')
        print(']')

#Testing
ACCURACY_THRESHOLD = 0.88
Addr = '/content/drive/MyDrive/CNNBrainAgeEstimation/nbml-challenge/Testing'
# Comma Separated Values File contain patient ID, Age and Sex 160 180
CSVFi = pandas.read_csv('/content/drive/MyDrive/CNNBrainAgeEstimation/nbml-challenge/Training-Ages.csv')
BufMat = MaSoSa(Addr, 5)
InMRsT, InMROnHoT = BufMat["InMRIs"], BufMat["InMRsOnHo"]
print('\n\nbefor add Canny: ', numpy.shape(InMRsT), end='\n\n')
InMRsT = list(numpy.reshape(InMRsT, (numpy.shape(InMRsT)[0], 256, 256)))
for i in range(numpy.shape(InMRsT)[0]):
    edges = cv2.Canny(numpy.uint8(InMRsT[i]), 550, 550, 3, L2gradient=True)
    InMRsT[i]= list(numpy.array(InMRsT[i])) + list(edges)
    #plt.imshow(edges)
    #plt.show()
    #pca.fit(InMRs[i])
    #InMRs[i] = pca.transform(InMRs[i])
print('\n\nafter add Canny: ', numpy.shape(InMRsT), '\n\n')#, numpy.shape(edges), '\n\n')
InMRsT = numpy.reshape(InMRsT, numpy.shape(InMRsT) + (1,))
InMROnHoT = numpy.argmax(InMROnHoT, axis=1)
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        PrLisT = Model.predict(InMRsT)
        Ac = accuracy_score(2 + InMROnHoT, 2 + numpy.argmax(PrLisT, axis = 1))
        print('\nAccuracy score: ', Ac, '\n\nTest matrix Shape: ', InMRsT.shape,
              '\nTest OneHot Shape: ', InMROnHoT.shape,
              '\n\nThe Time taht passed to creat input matrixs: ', ProTim()-StTi, '\n\n')
        if (logs.get('accuracy') > 0.80) and (logs.get('val_accuracy') > ACCURACY_THRESHOLD) and
             (logs.get('val_loss') < 2) or (Ac > 0.80):
            print("\nReached %2.2f%% val_accuracy, so stopping training!" %(ACCURACY_THRESHOLD*100))
            self.model.stop_training = True

# Training
# Initialization
Addr = '/content/drive/MyDrive/CNNBrainAgeEstimation/nbml-challenge/Training'
# input MRIs Matrix ((60x5), 256, 256)
# input MRIs One Hot
# Training Sub Path (Number of patients)
# MRI .dcm Picture
InMRs, InMROnHo = [], []
NuMode, StTi, MRICha = 0, 0, 0
BufMat = {}
TrPa = ""

print("\nThis is a program based on Keras API which implement the CNN (Convolutional Neural Network)",
      "\nYou can change its verious parameters!\n")
print("\nEnter number of mode: \n\nTrain with all samples\t   [0] \nTrain with some of samples [1]\n")
while True:
    NuMode = InCo()
    if NuMode == 0:
        BufMat = MaAlSa(Addr)
        InMRs, InMROnHo, MRICha = BufMat["InMRIs"], BufMat["InMRsOnHo"], BufMat["MRICha"]
        break
    elif NuMode == 1:
        BufMat = MaSoSa(Addr)
        InMRs, InMROnHo, MRICha = BufMat["InMRIs"], BufMat["InMRsOnHo"], BufMat["MRICha"]
        break
print('\n\nbefor add Canny: ', numpy.shape(InMRs), end='\n\n')
InMRs = list(numpy.reshape(InMRs, (numpy.shape(InMRs)[0],256,256)))
for i in range(numpy.shape(InMRs)[0]):
    edges = cv2.Canny(numpy.uint8(InMRs[i]), 550, 550, 3, L2gradient=True)
    InMRs[i]= list(numpy.array(InMRs[i])) + list(edges)
    #plt.imshow(edges)
    #plt.show()
    #pca.fit(InMRs[i])
    #InMRs[i] = pca.transform(InMRs[i])
print('\n\nafter add Canny: ', numpy.shape(InMRs), '\n\n')#, numpy.shape(edges), '\n\n')
InMRs = numpy.reshape(InMRs, numpy.shape(InMRs) + (1,))
#InMROnHo = numpy.argmax(InMROnHo, axis=1)
print('\n\ninput MRIs Matrix Shape: ', numpy.shape(InMRs))
print('\n\nOneHot Matrix Shape: \t ', numpy.shape(InMROnHo), '\n\n',
      '\nNumber of patients: ', len(TrPa), f'\tNumber of patients Samples: ' , numpy.shape(InMRs)[0],
      '\n\nThe Time taht passed to creat input matrixs: ', ProTim()-StTi , '\n')

# Validation Matrix
print('\nNumber of validation matrix samples:')
BufMat = MaSoSa('/content/drive/MyDrive/CNNBrainAgeEstimation/nbml-challenge/Validation')
print('\n\nbefor add Canny: ', numpy.shape(BufMat["InMRIs"]), end='\n\n')
BufMat["InMRIs"] = list(numpy.reshape(BufMat["InMRIs"], (numpy.shape(BufMat["InMRIs"])[0],256,256)))
for i in range(numpy.shape(BufMat["InMRIs"])[0]):
    edges = cv2.Canny(numpy.uint8(BufMat["InMRIs"][i]), 550, 20, 3, L2gradient=True)
    BufMat["InMRIs"][i]= list(numpy.array(BufMat["InMRIs"][i])) + list(edges)
    #pca.fit(InMRs[i])
    #InMRs[i] = pca.transform(InMRs[i])
print('\n\nafter add Canny: ', numpy.shape(BufMat["InMRIs"]))#, '\n\nEdges: ', numpy.shape(edges), '\n\n')
BufMat["InMRIs"] = numpy.reshape(BufMat["InMRIs"], numpy.shape(BufMat["InMRIs"]) + (1,))
print('\n\nAfter Reshape 187 line: ', numpy.shape(BufMat["InMRIs"]))
print('\n\nThe Time taht passed to creat validation matrixs: ', ProTim()-StTi, '\n\n')

# Build The Convolutional Neural Network By keras ((Creat model phase), Work with channels_last Data)
Model = keras.Sequential()
#clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
while True:
    print("\nModel will created with Keras version: {keras.__version__} and {keras.backend()} Backend!\n")
    print("\nThe model can train in 3 modes!\numpylease enter the number of mode (Just one number!):")
    print("\nTraining with default model [0]\nTraining with custom model  [1]\nTraining with saved model   [2]")
    NuMode = InCo()
    if NuMode == 0:
        print('\n\n')
        Model.add(keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(2,2), activation='relu',
                                      input_shape=(512,256,1)))
        Model.add(keras.layers.AveragePooling2D(pool_size=(3,3), strides=(2,2)))
        Model.add(keras.layers.Conv2D(filters=12, kernel_size=(3,3), strides=(2,2), activation='relu'))
        Model.add(keras.layers.AveragePooling2D(pool_size=(3,3), strides=(2,2)))
        Model.add(keras.layers.Conv2D(filters=8, kernel_size=(3,3), strides=(2,2), activation='relu'))
        Model.add(keras.layers.MaxPooling2D(pool_size=(3,3), strides=(1,1)))
        Model.add(keras.layers.Flatten())
        Model.add(keras.layers.Dense(8000, activation='relu'))
        Model.add(keras.layers.Dropout(0.5))
        Model.add(keras.layers.Dense(4000, activation='relu'))
        Model.add(keras.layers.Dropout(0.5))
        Model.add(keras.layers.Dense(2000, activation='relu'))
        Model.add(keras.layers.Dropout(0.5))
        Model.add(keras.layers.Dense(1000, activation='relu'))
        Model.add(keras.layers.Dropout(0.5))
        Model.add(keras.layers.Dense(500, activation='relu'))
        Model.add(keras.layers.Dropout(0.3))
        Model.add(keras.layers.Dense(250, activation='relu'))
        Model.add(keras.layers.Dense(6, activation='softmax'))
        keras.optimizers.Adam(learning_rate=0.0000022, beta_1=0.9, beta_2=0.999, amsgrad=False)
        print('\n\nbefor add Canny: ', numpy.shape(InMRs), end='\n\n')
        # Compile CNN Model
        Model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
        # Start time to train model
        StTi = ProTim()
        Model.fit(InMRs, InMROnHo, batch_size=InMRs.shape[0], epochs=200,
                  validation_data=(BufMat["InMRIs"], BufMat["InMRsOnHo"]),
                  callbacks=[myCallback()], verbose = 2)
        break
    elif NuMode == 1:
        ParArr = numpy.int16(numpy.empty((8)))
        IsFiLa = 1
        while True:
            print("\n\numpylease enter your desired layer number (Number in front of each layer):",
                  "\nConv2D \t\t [0] \nAveragePooling2D [1] \nFlatten \t [2] \nDense \t\t [3] \nDropout \t [4]\n",
                  "\nNext settings \t [5]")
            NuMode = InCo()
            if NuMode == 0:
                print("\n\nNumber of filters")
                ParArr[0] = InCo()
                print("\nFirst dimension of kernel size")
                ParArr[1] = InCo()
                print("\nSecond dimension of kernel size")
                ParArr[2] = InCo()
                print("\nFirst dimension of strides")
                ParArr[3] = InCo()
                print("\nSecond dimension of strides")
                ParArr[4] = InCo()
                if IsFiLa == 1:
                    IsFiLa = 0
                    print("\nFirst dimension of input shape")
                    ParArr[5] = InCo()
                    print("\nSecond dimension of input shape")
                    ParArr[6] = InCo()
                    print("\nThird  dimension of input shape")
                    ParArr[7] = InCo()
                    print(type(ParArr), '\n\n', ParArr, '\n\n')
                    Model.add(keras.layers.Conv2D(filters=ParArr[0], kernel_size=(ParArr[1],ParArr[2]), strides=(ParArr[3],ParArr[4]),
                                                   activation='relu', input_shape=(ParArr[5],ParArr[6],ParArr[7])))
                    continue
                Model.add(keras.layers.Conv2D(filters=ParArr[0], kernel_size=(ParArr[1],ParArr[2]), strides=(ParArr[3],ParArr[4]),activation='relu'))
                continue
            elif NuMode == 1:
                print("\n\nFirst dimension of pool size")
                ParArr[0] = InCo()
                print("\nSecond dimension of pool size")
                ParArr[1] = InCo()
                print("\nFirst dimension of strides")
                ParArr[2] = InCo()
                print("\nSecond dimension of strides")
                ParArr[3] = InCo()
                Model.add(keras.layers.AveragePooling2D(pool_size=(ParArr[0],ParArr[1]), strides=(ParArr[2],ParArr[3])))
                continue
            elif NuMode == 2:
                Model.add(keras.layers.Flatten())
                continue
            elif NuMode == 3:
                print("\n\nNumber of neurons")
                ParArr[0] = InCo()
                Model.add(keras.layers.Dense(ParArr[0], activation='relu'))
                continue
            elif NuMode == 4:
                print("\n\nNoise reduction rate")
                ParArr[0] = InCo()
                Model.add(keras.layers.Dropout(ParArr[0]/10))
                continue
            elif NuMode == 5:
                while True:
                    print("\n\numpylease enter your desired option number (Number in front of each line):",
                      "\n\nLaDens \t\t [0] \nAdam setting \t [1]\nCompile model \t [2]\nFit (training) \t [3]",
                      "\n\nFor return, enter integer bigger than [3] ")
                    NuMode = InCo()
                    if NuMode == 0:
                        print("\n\nNumber of last neurons (number of classes)")
                        ParArr[0] = InCo()
                        Model.add(keras.layers.Dense(ParArr[0], activation='softmax'))
                        continue
                    elif NuMode == 1:
                        print("\n\nValue of learning rate")
                        ParArr[0] = InCo()
                        print("\nValue of beta 1")
                        ParArr[1] = InCo()
                        print("\nValue of beta 2")
                        ParArr[2] = InCo()
                        keras.optimizers.Adam(learning_rate=(ParArr[0]/10), beta_1=(ParArr[1]/10), beta_2=(ParArr[2]/10), amsgrad=False)
                        continue
                    elif NuMode == 2:
                        Model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
                        continue
                    elif NuMode == 3:
                        print("\n\nNumber of batch size")
                        ParArr[0] = InCo()
                        print("\nNumber of epoch")
                        ParArr[1] = InCo()
                        print("\nValue of validation split")
                        ParArr[2] = InCo()
                        print('\n\n')
                        StTi = ProTim()
                        Model.fit(InMRs, InMROnHo, batch_size=ParArr[0], epochs=ParArr[1], validation_data=(BufMat["InMRIs"], BufMat["InMRsOnHo"]),
                                  callbacks=[myCallback()], verbose = 2)
                    break
                if NuMode == 3:
                    break
        break
    elif NuMode == 2:
        #/content/New80.h5
        Model = keras.models.load_model(input("\n\nEnter file path to load model: "))
        print("\n\nNumber of resume epoch")
        NuReEp = InCo()
        print("\n\nNumber of epochs")
        NuMode = InCo()
        print("\n\n")
        StTi = ProTim()
        print('\n\nbefor add Canny: ', numpy.shape(InMRs), end = '\n\n')
        InMRs = list(numpy.reshape(InMRs, (462,256,256)))
        for i in range(numpy.shape(InMRs)[0]):
            edges = cv2.Canny(numpy.uint8(InMRs[i]), 550, 20, 3, L2gradient = True)
            InMRs[i]= list(numpy.array(InMRs[i])) + list(numpy.uint16(edges))
            #pca.fit(InMRs[i])
            #InMRs[i] = pca.transform(InMRs[i])
        print('\n\nafter add Canny: ', numpy.shape(InMRs[0]))#, '\n\nEdges: ', numpy.shape(edges), '\n\n')
        InMRs = numpy.reshape(InMRs, numpy.shape(InMRs) + (1,))
        Model.fit(InMRs, InMROnHo, batch_size=InMRs.shape[0], epochs=NuReEp+NuMode, initial_epoch=NuReEp,
                  validation_data = (BufMat["InMRIs"], BufMat["InMRsOnHo"]), callbacks=[myCallback()],
                  verbose = 2)
        """# CNN Feature extract
        extract = keras.Model(Model.inputs, Model.layers[4].output)
        numpy_image_features = extract.predict(InMRs)
        print(numpy.shape(numpy_image_features),extract.layers)
        # SVM
        image_scene_labels = [(list(i).index(1)+2)*10 for i in InMROnHo[:]]
        X_train, y_train = numpy_image_features, image_scene_labels
        print(X_train.shape, numpy.shape(y_train))
        #clf.fit(X_train, y_train)
        y_pred = clf.predict(X_train)"""
        OnSaCl = [(list(i).index(1)+2)*10 for i in InMROnHo[:]]
        PrLis = Model.predict(InMRs)
        PrLis = numpy.argmax(PrLis, axis=1)
        print(PrLis)
        plt.figure(figsize = (9,12))
        HitTags=[20,30,40,50,60]
        sn.heatmap(confusion_matrix(y_true=OnSaCl, y_pred=(PrLis+2)*10),yticklabels=HitTags,
                   xticklabels = HitTags, square=True, cmap="OrRd")
        plt.show()
        break
    else:
        input("\n\nHave a good day!")
        exit()

# End time of training
print("\nEnd of Training!\n\n")
Model.summary()
print("\n\nThe time that passed for tarining:", (ProTim()-StTi)/60, "Min")
try:
    print('\n\nGPU Information:\n%s' %nvsmi.get_gpu_processes(), '\n\n')
except:
    print('\nGPU Not Detected!\n')


# Prediction phase
InMRs, InMROnHo = [], []
print("\n\nPrediction modes:\n\n"
      "\nPrediction samples       [0]"
      "\nPrediction some samples  [1]"
      "\nPrediction a sample      [2]"
      "\n\nFor exit, enter integer bigger than [2]")
NuMode = InCo()
if NuMode == 0:
    Addr = '/content/drive/MyDrive/CNNBrainAgeEstimation/nbml-challenge/Testing'
    BufMat = MaAlSa(Addr)
    InMRs, InMROnHo = BufMat["InMRIs"], BufMat["InMRsOnHo"]
    OnSaCl = numpy.argmax(InMROnHo, axis=1)
    PrLis = numpy.argmax(Model.predict(InMRs), axis=1)
    print(3*'\n\n','Test matrix Shape: ', InMRs.shape, '\nTest OneHot Shape: ', InMROnHo.shape,
          '\n\nThe Time taht passed to creat input matrixs: ', ProTim()-StTi, '\n\nClassification Report:\n\n',
          classification_report(2+OnSaCl, 2+PrLis, zero_division=1))
    CoMa = confusion_matrix(OnSaCl,PrLis)
    CoMaSh(CoMa)
elif NuMode == 1:
    Addr = '/content/drive/MyDrive/CNNBrainAgeEstimation/nbml-challenge/Testing'
    BufMat = MaSoSa(Addr)
    InMRs, InMROnHo = BufMat["InMRIs"], BufMat["InMRsOnHo"]
    print('\n\nbefor add Canny: ', numpy.shape(InMRs), end = '\n\n')
    InMRs = list(numpy.reshape(InMRs, (numpy.shape(InMRs)[0], 256, 256)))
    for i in range(numpy.shape(InMRs)[0]):
        edges = cv2.Canny(numpy.uint8(InMRs[i]), 550, 20, 3, L2gradient = True)
        InMRs[i] = list(numpy.array(InMRs[i])) + list(edges)
        #pca.fit(InMRs[i])
        #InMRs[i] = pca.transform(InMRs[i])
    print('\n\nafter add Canny: ', numpy.shape(InMRs[0]))#, '\n\nEdges: ', numpy.shape(edges), '\n\n')
    InMRs = numpy.reshape(InMRs, numpy.shape(InMRs) + (1,))
    OnSaCl = numpy.argmax(InMROnHo, axis=1)
    PrLis = numpy.argmax(Model.predict(InMRs), axis=1)
    print('\n\nTest matrix Shape: ', InMRs.shape, '\nTest OneHot Shape: ', InMROnHo.shape,
          '\n\nThe Time taht passed to creat input matrixs: ', ProTim()-StTi,
          '\n\nClassification Report:\n\n', classification_report(2+OnSaCl, 2+PrLis, zero_division=1))
    CoMa = confusion_matrix(OnSaCl,PrLis)
    CoMaSh(CoMa)
    OnSaCl = [(list(i).index(1)+2)*10 for i in InMROnHo[:]]
    PrLis = Model.predict(InMRs)
    PrLis = numpy.argmax(PrLis, axis=1)
    print(PrLis)
    plt.figure(figsize = (9,12))
    HitTags=[20,30,40,50,60]
    sn.heatmap(confusion_matrix(y_true=OnSaCl, y_pred=(PrLis+2)*10), yticklabels = HitTags,
               xticklabels = HitTags, square=True, cmap="OrRd")
    plt.show()
    """
    # CNN Feature extract
    extract = keras.Model(Model.inputs, Model.layers[4].output)
    numpy_image_features = extract.predict(InMRs)
    print(numpy.shape(numpy_image_features),extract.layers)
    # SVM
    image_scene_labels = [(list(i).index(1)+2)*10 for i in InMROnHo[:]]
    X_test, y_test = numpy_image_features, image_scene_labels
    print(X_test.shape, numpy.shape(y_test))
    y_pred = clf.predict(X_test)
    print(classification_report(y_true=y_test, y_pred=y_pred, target_names=['20','30','40','50','60']))
    print(confusion_matrix(y_true=y_test, y_pred=y_pred))"""
elif NuMode == 2:
    PrSa = dcmread(input("\nEnter file path to load predict sample: "))
    print("\n\nClass:\n\n", Model.predict(numpy.reshape(PrSa.pixel_array, (1, 256, 256, 1))) * 100,
          "\numpyatient ID: ", PrSa.PatientName)
# Saving model
if input("\n\nDo you want to save your whole model (Enter 1 instead [Yes]): ") == '1':
    Model.save(input("\nEnter file path to save the model: "))

# End of Timer (The time that program is runnning)
print("\n\nThe time that passed: ", ProTim()-ST)

# **Dataset Unpacking**

In [ ]:
import tarfile

fname = "/content/drive/MyDrive/CNNBrainAgeEstimation/IXI-T1.tar"

if fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
    tar.extractall()
    tar.close()
"""
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

example_filename = os.path.join('/content/', 'IXI035-IOP-0873-T1.nii.gz')
img = nib.load(example_filename)
print(f'shape of T1-Mri: %s' %str(np.shape(img.get_data())))
for i in range(100,150):
  print(i)
  plt.imshow(img.get_fdata()[:][:][i])
  plt.show()
"""
"""
print(img.Version)
plt.imshow((img.get_data())[:][:][140])
plt.show()"""

/tmp/ipython-input-3264435891.py:7: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall()


'\nprint(img.Version)\nplt.imshow((img.get_data())[:][:][140])\nplt.show()'

# **Costumized VGG16 Testing**

In [ ]:
from keras.wrappers import SKLearnRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import confusion_matrix
#from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
#from sklearn.svm import SVC
from tensorflow import keras
from keras.models import Model as KerasModel
from pydicom import dcmread
import nibabel as nib
import random
import seaborn as sn
import pandas
import numpy
import nvsmi
import cv2
import time
import math
import os

# well-defined initial state
#

# Start of Timer (Data preparation phase)
ST = time.perf_counter()

numpy.random.seed(23)
# input Control
def InCo():
    InVa = 0
    while True:
        try:
            InVa = int(input("\nEnter Number: "))
            if InVa >= 0:
                return InVa
            else:
                print("\ninput is not positive integer!\n")
        except:
            print("\ninput is not integer!\n")

# Return current time (Program Time)
def ProTim():
    return time.perf_counter()



def CoMaSh(CoM):
    print("\n\nConfusion matrix:\n\n     %20s"  %"2 3 4 5 6 7 8".replace(' ', '    '))
    for i in range(numpy.shape(CoMa)[0]):
        print('\n', 2+i, ' [', end = '')
        for j in range(numpy.shape(CoMa)[0]):
            if j == numpy.shape(CoMa)[0]-1:
                print("%s" %(CoMa[i][j]), end = '')
                break
            print("%-5s" %(CoMa[i][j]), end = '')
        print(']')


def baseline_model():
  model = keras.Sequential()
  model.add(keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(2,2), activation='relu', input_shape=(512,256,1)))
  model.add(keras.layers.AveragePooling2D(pool_size=(3,3), strides=(2,2)))
  model.add(keras.layers.Conv2D(filters=12, kernel_size=(3,3), strides=(2,2), activation='relu'))
  model.add(keras.layers.AveragePooling2D(pool_size=(3,3), strides=(2,2)))
  model.add(keras.layers.Conv2D(filters=8, kernel_size=(3,3), strides=(2,2), activation='relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=(3,3), strides=(1,1)))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(8000, activation='relu'))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(4000, activation='relu'))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(2000, activation='relu'))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(1000, activation='relu'))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(500, activation='relu'))
  model.add(keras.layers.Dropout(0.3))
  model.add(keras.layers.Dense(250, activation='relu'))
  model.add(keras.layers.Dense(1))
  LeaRa = 0.001
  Opt = keras.optimizers.Adam(learning_rate=LeaRa, beta_1=0.9, beta_2=0.999, amsgrad=True)
  model.compile(loss='mean_squared_error', optimizer= Opt, metrics=[tf.keras.losses.MeanSquaredError()])
  return model

# Main Start
# Initialization
# input MRIs Matrix ((60x5), 256, 256)
# input MRIs One Hot
# Training Sub Path (Number of patients)
# MRI .dcm Picture
InMRs, InMROnHo = [], []
NuMode, StTi, MRICha = 0, 0, 0
BufMat = {}
TrPa = ""

LisFiles = os.listdir('/content')
CSVFi = pandas.read_csv('/content/drive/MyDrive/CNNBrainAgeEstimation/IDSEAG.csv')
Age20, Age30, Age40 = [], [], []
Age50, Age60 , Age70 = [], [], []
Age80 = []
NuSa = None
# Number of Patients Samples use to Creat input MRIs Matrix
NuSa = int(input("\n\nEnter number of samples to use: "))
for i in LisFiles:
    example_filename = os.path.join('/content/', i)
    """
    for j in range((len(example_Subfilnam)//2)+(-NuSa//2),(len(example_Subfilnam)//2)+(-NuSa//2)+NuSa):
        for k in range(0,len(CSVFi.Subject_ID)):
          MRdcm = dcmread(example_filename+'/'+example_Subfilnam[j])
          if example_Subfilnam[j].endswith('.dcm') and str(CSVFi.Subject_ID[k]) in MRdcm.PatientName:
            if (20 <= CSVFi.Age[k]) and (CSVFi.Age[k] < 30):
                Age20.append([MRdcm.pixel_array, CSVFi.Age[k]])
            elif (30 <= CSVFi.Age[k]) and (CSVFi.Age[k] < 40):
                Age30.append([MRdcm.pixel_array, CSVFi.Age[k]])
            elif (40 <= CSVFi.Age[k]) and (CSVFi.Age[k] < 50):
                Age40.append([MRdcm.pixel_array, CSVFi.Age[k]])
            elif (50 <= CSVFi.Age[k]) and (CSVFi.Age[k] < 60):
                Age50.append([MRdcm.pixel_array, CSVFi.Age[k]])
            elif (60 <= CSVFi.Age[k]) and (CSVFi.Age[k] < 70):
                Age60.append([MRdcm.pixel_array, CSVFi.Age[k]])
            elif (70 <= CSVFi.Age[k]) and (CSVFi.Age[k] < 80):
                Age70.append([MRdcm.pixel_array, CSVFi.Age[k]])
            elif (80 <= CSVFi.Age[k]) and (CSVFi.Age[k] < 90):
                Age80.append([MRdcm.pixel_array, CSVFi.Age[k]])
            if not ((20 <= CSVFi.Age[k]) and (CSVFi.Age[k] < 90)):
                break
    """
    for j in range(len(CSVFi.IXI_ID)):
      if i.endswith('.gz') and CSVFi.IXI_ID[j] == int(i.split('-')[0][3:]):
        img = nib.load(example_filename)
        if (20 <= CSVFi.AGE[j]) and (CSVFi.AGE[j] < 30):
            Age20.append([cv2.resize(img.get_fdata()[:,130,:], (256,256)), CSVFi.AGE[j]])
        elif (30 <= CSVFi.AGE[j]) and (CSVFi.AGE[j] < 40):
            Age30.append([cv2.resize(img.get_fdata()[:,130,:], (256,256)), CSVFi.AGE[j]])
        elif (40 <= CSVFi.AGE[j]) and (CSVFi.AGE[j] < 50):
            Age40.append([cv2.resize(img.get_fdata()[:,130,:], (256,256)), CSVFi.AGE[j]])
        elif (50 <= CSVFi.AGE[j]) and (CSVFi.AGE[j] < 60):
            Age50.append([cv2.resize(img.get_fdata()[:,130,:], (256,256)), CSVFi.AGE[j]])
        elif (60 <= CSVFi.AGE[j]) and (CSVFi.AGE[j] < 70):
            Age60.append([cv2.resize(img.get_fdata()[:,130,:], (256,256)), CSVFi.AGE[j]])
        elif (70 <= CSVFi.AGE[j]) and (CSVFi.AGE[j] < 80):
            Age70.append([cv2.resize(img.get_fdata()[:,130,:], (256,256)), CSVFi.AGE[j]])
        elif (80 <= CSVFi.AGE[j]) and (CSVFi.AGE[j] < 90):
            Age80.append([cv2.resize(img.get_fdata()[:,130,:], (256,256)), CSVFi.AGE[j]])
        if not ((20 <= CSVFi.AGE[j]) and (CSVFi.AGE[j] < 90)):
            break
TraiImags = []
TraiLabel = []
TestImags = []
TestLabel = []
ValiImags = []
ValiLabel = []

Percent = 70
pool = [Age20, Age30, Age40, Age50, Age60, Age70, Age80]
for i in pool:
    if i == []:
        continue
    random.shuffle(i)
    TraNum = round((len(i)*Percent)/100)
    VaTeNum = round((len(i)*(100-Percent))/100)
    ValNum = round(VaTeNum/4.5)
    #Labels = [0]*7
    #Labels[pool.index(i)] = 1
    for j in range(0, TraNum):
      TraiImags.append(i[j][0])
      TraiLabel.append(i[j][1])
    #print("shape of: ", numpy.shape(i[0][0]))
    if i[TraNum+ValNum:] == []:
      continue
    for j in range(TraNum, TraNum+ValNum):
      ValiImags.append(i[j][0])
      ValiLabel.append(i[j][1])
    for j in range(TraNum + ValNum, len(i)):
      TestImags.append(i[j][0])
      TestLabel.append(i[j][1])
print("Train: %s \nTest: %s \nValid: %s" %(numpy.shape(TraiImags),
                                           numpy.shape(TestImags),
                                           numpy.shape(ValiImags)))
input()
del Age20,Age30,Age40,Age50,Age60,Age70,Age80

ACCURACY_THRESHOLD = 0.88
# Comma Separated Values File contain patient ID, Age and Sex 160 180
InMRs, InMROnHoTr = TraiImags, numpy.array(TraiLabel)

NuMode, StTi, MRICha = 0, 0, 0
BufMat = ValiImags

InMRsT, InMROnHoT = TestImags, numpy.array(TestLabel)

print('\n\nbefor add Canny: ', numpy.shape(InMRs), end='\n\n')
InMRs = list(numpy.reshape(InMRs, (numpy.shape(InMRs)[0],256,256)))
for i in range(numpy.shape(InMRs)[0]):
    edges = cv2.Canny(numpy.uint8(InMRs[i][:]), 120, 220, 3, L2gradient=True)
    InMRs[i]= list(numpy.array(InMRs[i])) + list(edges)
    #pca.fit(InMRs[i])
    #InMRs[i] = pca.transform(InMRs[i])
print('\n\nafter add Canny: ', numpy.shape(InMRs[0]))#, '\n\nEdges: ', numpy.shape(edges), '\n\n')
InMRsTr = numpy.reshape(InMRs,(numpy.shape(InMRs)[0],numpy.shape(InMRs)[1],numpy.shape(InMRs)[2],1))
#print(f'shape of T1-Mri: %s' %str(numpy.shape(img.get_fdata())))
plt.imshow(numpy.reshape(InMRs[0],(512,256)), cmap='gray')
plt.show()
del InMRs

print('\n\nbefor add Canny: ', numpy.shape(InMRsT), end='\n\n')
InMRsT = list(numpy.reshape(InMRsT, (numpy.shape(InMRsT)[0],256,256)))
for i in range(numpy.shape(InMRsT)[0]):
    edges = cv2.Canny(numpy.uint8(InMRsT[i][:]), 120, 220, 3, L2gradient=True)
    InMRsT[i]= list(numpy.array(InMRsT[i])) + list(edges)
    #pca.fit(InMRs[i])
    #InMRs[i] = pca.transform(InMRs[i])
print('\n\nafter add Canny: ', numpy.shape(InMRsT), '\n\n')#, numpy.shape(edges), '\n\n')
InMRsT = numpy.reshape(InMRsT,(numpy.shape(InMRsT)[0],numpy.shape(InMRsT)[1],numpy.shape(InMRsT)[2],1))
OnSaClT = InMROnHoT

class myCallback(keras.callbacks.Callback):
    global ShortError
    def on_epoch_end(self, epoch, logs=None):
        global ShortError
        PrLisT = Model.predict(InMRsT)
        RMSE = math.sqrt(mean_squared_error(OnSaClT,PrLisT, squared = True))
        if RMSE < ShortError:
          ShortError = RMSE
        print('\n\nTest matrix Shape: ', numpy.shape(InMRsT), '\nTest OneHot Shape: ', numpy.shape(InMROnHoT),
              '\n\nRoot Mean_squared_error: ', RMSE, '\n\nShortest Error: ', ShortError, '\n\n')
        if RMSE < 13:
          print("\n\nAt last! I find it!\n\n")
          input("\nWoo ha ha haaaaa...\n")
          self.model.stop_training = True
ShortError = 10000000
#Validation Matrix
print('\nNumber of validation matrix samples:')
print('\n\nbefor add Canny: ', numpy.shape(BufMat), end='\n\n')
BufMat = list(numpy.reshape(BufMat, (numpy.shape(BufMat)[0],256,256)))
for i in range(numpy.shape(BufMat)[0]):
    edges = cv2.Canny(numpy.uint8(BufMat[i]), 120, 220, 3, L2gradient=True)
    BufMat[i]= list(numpy.array(BufMat[i])) + list(edges)
    #pca.fit(InMRs[i])
    #InMRs[i] = pca.transform(InMRs[i])
print('\n\nafter add Canny: ', numpy.shape(BufMat))#, '\n\nEdges: ', numpy.shape(edges), '\n\n')
BufMat = numpy.reshape(BufMat,(numpy.shape(BufMat)[0],numpy.shape(BufMat)[1],numpy.shape(BufMat)[2],1))
print('\n\nThe Time taht passed to creat validation matrixs: ', ProTim()-StTi, '\n\n')
InMRsTr = list(InMRsTr)
InMRsTr.extend(list(BufMat))
InMROnHoTr = list(InMROnHoTr)
InMROnHoTr.extend(list(ValiLabel))
for i in range(0,20):
  NuMode, StTi, MRICha = 0, 0, 0
  print("\nThis is a program based on Keras API which implement the CNN (Convolutional Neural Network)",
        "\nYou can change its verious parameters!\n")
  print("\nEnter number of mode: \n\nTrain with all samples\t   [0] \nTrain with some of samples [1]\n")

  print('\n\ninput MRIs Matrix Shape: ', numpy.shape(InMRsTr))
  print('\n\nOneHot Matrix Shape: \t ', numpy.shape(InMROnHoTr), '\n\n',
        '\nNumber of patients: ', f'\tNumber of patients Samples: ' , numpy.shape(InMRsTr)[0],
        '\n\nThe Time taht passed to creat input matrixs: ', ProTim()-StTi , '\n')

  # Build The Convolutional Neural Network By keras ((Creat model phase), Work with channels_last Data)
  # define a new model with random weights and 10 classes
  ModelVGG16 = VGG16(weights = 'imagenet', classes = 1000, include_top = False, input_shape = (512, 256, 3))
  ModelVGG16.summary()

  inp2 = ModelVGG16.input
  new_layer2 = Dense(10, activation='softmax', name='my_dense_2')
  out2 = new_layer2(Flatten()(ModelVGG16.output))

  CosVGG16 = KerasModel(inp2, out2)

  #clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
  while True:
      print("\nThe model can train in 2 modes!\nplease enter the number of mode (Just one number!):\n")
      print("\nTraining with default model [0]\nTraining with saved model   [1]")
      NuMode = 0 #InCo()
      if NuMode == 0:
          print('\n\n')
          """Model.add(keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(2,2), activation='relu', input_shape=(512,256,1)))
          Model.add(keras.layers.AveragePooling2D(pool_size=(3,3), strides=(2,2)))
          Model.add(keras.layers.Conv2D(filters=12, kernel_size=(3,3), strides=(2,2), activation='relu'))
          Model.add(keras.layers.AveragePooling2D(pool_size=(3,3), strides=(2,2)))
          Model.add(keras.layers.Conv2D(filters=8, kernel_size=(3,3), strides=(2,2), activation='relu'))
          Model.add(keras.layers.MaxPooling2D(pool_size=(3,3), strides=(1,1)))
          Model.add(keras.layers.Flatten())
          Model.add(keras.layers.Dense(520, activation='relu'))
          Model.add(keras.layers.Dropout(0.5))
          Model.add(keras.layers.Dense(260, activation='relu'))
          Model.add(keras.layers.Dropout(0.5))
          Model.add(keras.layers.Dense(130, activation='relu'))
          Model.add(keras.layers.Dropout(0.5))
          Model.add(keras.layers.Dense(65, activation='relu'))
          Model.add(keras.layers.Dropout(0.5))
          Model.add(keras.layers.Dense(32, activation='relu'))
          Model.add(keras.layers.Dropout(0.3))
          Model.add(keras.layers.Dense(16, activation='relu'))
          Model.add(keras.layers.Dense(1))
          LeaRa = float(input("learning rate: "))
          Opt = keras.optimizers.Adam(learning_rate=LeaRa, beta_1=0.9, beta_2=0.999, amsgrad=False)

          # Compile CNN Model
          #Model.compile(loss='categorical_crossentropy', optimizer= Opt, metrics=['accuracy'])
          Model.compile(loss='mean_squared_error', optimizer= Opt, metrics=[tf.keras.losses.MeanSquaredError()])"""
          # Start time to train model
          StTi = ProTim()#numpy.shape(InMRsTr)[0]
          ImageBuf = numpy.zeros((451, 512, 256, 3))
          InMRsTr = numpy.array(InMRsTr)
          ImageBuf[:,:,:,0] = InMRsTr[:,:,:,0]
          ImageBuf[:,:,:,1] = InMRsTr[:,:,:,0]
          ImageBuf[:,:,:,2] = InMRsTr[:,:,:,0]
          #CosVGG16.fit(ImageBuf, numpy.array(InMROnHoTr), batch_size= 5, epochs=900,
          #             validation_split = 0.088, callbacks = [myCallback()], verbose = 2)

          # split into input (X) and output (Y) variables
          # define base model
          # evaluate model
          #estimator = KerasRegressor(build_fn=baseline_model, epochs=500, batch_size=numpy.shape(InMRs)[0], verbose=2)
          #kfold = KFold(n_splits=5)
          #results = cross_val_score(estimator, numpy.array(InMRs), numpy.array(InMROnHo), cv=kfold)
          #print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))
          break
      elif NuMode == 1:
          # /content/New80.h5
          Model = keras.models.load_model(input("\n\nEnter file path to load model: "))
          print("\n\nNumber of resume epoch")
          NuReEp = InCo()
          print("\n\nNumber of epochs")
          NuMode = InCo()
          print("\n\n")
          StTi = ProTim()
          print('\n\nbefor add Canny: ', numpy.shape(InMRs), end='\n\n')
          InMRs = list(numpy.reshape(InMRs, (462,256,256)))
          for i in range(numpy.shape(InMRs)[0]):
              edges = cv2.Canny(numpy.uint8(InMRs[i][:]), 120, 220, 3, L2gradient=True)
              InMRs[i]= list(numpy.array(InMRs[i])) + list(numpy.uint16(edges))
              #pca.fit(InMRs[i])
              #InMRs[i] = pca.transform(InMRs[i])
          print('\n\nafter add Canny: ', numpy.shape(InMRs[0]))#, '\n\nEdges: ', numpy.shape(edges), '\n\n')
          InMRs = numpy.reshape(InMRs,(numpy.shape(InMRs)[0],numpy.shape(InMRs)[1],numpy.shape(InMRs)[2],1))
          Model.fit(InMRs, InMROnHo, batch_size=InMRs.shape[0], epochs=NuReEp+NuMode, initial_epoch=NuReEp, validation_data=(BufMat["InMRIs"], BufMat["InMRsOnHo"]),
                    callbacks=[myCallback()], verbose = 2)
          """
          # CNN Feature extract
          extract = keras.Model(Model.inputs, Model.layers[4].output)
          numpy_image_features = extract.predict(InMRs)
          print(numpy.shape(numpy_image_features),extract.layers)
          # SVM
          image_scene_labels = [(list(i).index(1)+2)*10 for i in InMROnHo[:]]
          X_train, y_train = numpy_image_features, image_scene_labels
          print(X_train.shape, numpy.shape(y_train))
          #clf.fit(X_train, y_train)
          y_pred = clf.predict(X_train)
          """
          OnSaCl = [(list(i).index(1)+2)*10 for i in InMROnHo[:]]
          PrLis = Model.predict(InMRs)
          PrLis = numpy.argmax(PrLis, axis=1)
          print(PrLis)
          plt.figure(figsize = (9,12))
          HitTags=[20,30,40,50,60]
          sn.heatmap(confusion_matrix(y_true=OnSaCl, y_pred=(PrLis+2)*10),yticklabels=HitTags, xticklabels = HitTags, square=True, cmap="OrRd")
          plt.show()
          break
      else:
          input("\n\nHave a good day!")
          exit()

  # End time of training
  print("\nEnd of Training!\n\n")
  CosVGG16.summary()
  print("\n\nThe time that passed for tarining: ", ProTim()-StTi, '\n\n')
  print('\n\nGPU Information:\n%s' %nvsmi.get_gpu_processes(), '\n\n')

  # Prediction phase
  InMRs, InMROnHo = [], []
  print("\n\npredict modes: \n\npredict samples \t[0] \npredict some samples \t[1]",
        "\npredict a sample \t[2] \n\nFor exit, enter integer bigger than [2]")
  NuMode = 1#InCo()
  if NuMode == 0:
      BufMat = MaAlSa(Addr)
      InMRs, InMROnHo = BufMat["InMRIs"], BufMat["InMRsOnHo"]
      OnSaCl = numpy.argmax(numpy.array(InMROnHo), axis=1)
      PrLis = numpy.argmax(CosVGG16.predict(InMRs), axis=1)
      print(3*'\n\n','Test matrix Shape: ', InMRs.shape, '\nTest OneHot Shape: ', InMROnHo.shape,
            '\n\nThe Time taht passed to creat input matrixs: ', ProTim()-StTi, '\n\nClassification Report:\n\n',
            classification_report(2+OnSaCl, 2+PrLis, zero_division=1))
      CoMa = confusion_matrix(OnSaCl,PrLis)
      CoMaSh(CoMa)
  elif NuMode == 1:
      InMRs, InMROnHo = InMRsT, InMROnHoT
      InMRs = numpy.reshape(InMRs,(numpy.shape(InMRs)[0],numpy.shape(InMRs)[1],numpy.shape(InMRs)[2],1))

      OnSaCl = numpy.array(InMROnHo)
      PrLis = CosVGG16.predict(numpy.array(InMRs))
      print('\n\nTest matrix Shape: ', numpy.shape(InMRs), '\nTest OneHot Shape: ', numpy.shape(InMROnHo),
            '\n\nThe Time taht passed to creat input matrixs: ', ProTim()-StTi)
      #results = cross_val_score(estimator, numpy.array(InMRs), numpy.array(OnSaCl), cv=kfold)
      #print("Baseline: %.2f (%.2f) MSE\n %.2f" % (results.mean(), results.std(),math.sqrt(mean_squared_error(OnSaCl,PrLis, squared = True))))
      print("\n\nPredicted:\n\n", numpy.reshape(PrLis, numpy.shape(OnSaCl)), "\n\nOriginal:\n\n", OnSaCl,'\n\nRand: ', i,
             '\n\nShortest Error: ', ShortError, '\n\n')
      """
      # CNN Feature extract
      extract = keras.Model(Model.inputs, Model.layers[4].output)
      numpy_image_features = extract.predict(InMRs)
      print(numpy.shape(numpy_image_features),extract.layers)
      # SVM
      image_scene_labels = [(list(i).index(1)+2)*10 for i in InMROnHo[:]]
      X_test, y_test = numpy_image_features, image_scene_labels
      print(X_test.shape, numpy.shape(y_test))
      y_pred = clf.predict(X_test)
      print(classification_report(y_true=y_test, y_pred=y_pred, target_names=['20','30','40','50','60']))
      print(confusion_matrix(y_true=y_test, y_pred=y_pred))"""
  elif NuMode == 2:
      PrSa = dcmread(input("\nEnter file path to load predict sample: "))
      print("\n\nClass:\n\n", CosVGG16.predict(numpy.reshape(PrSa.pixel_array,(1,256,256,1)))*100,
            "\numpyatient ID: ", PrSa.PatientName)
  # Saving model
  if input("\n\nDo you want to save your whole model (Enter 1 instead [Yes]): ") == '1':
      CosVGG16.save(input("\nEnter file path to save the model: "))

  # End of Timer (The time that program is runnning)
  print("\n\nThe time that passed: ", ProTim()-ST)

# **VGG16 Transfer learning**

In [ ]:
# example of loading the vgg16 model
from keras.applications.vgg16 import VGG16

# define a new model with random weights and 10 classes

model = VGG16(weights='imagenet', classes=1000)
# summarize the model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import applications, optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Input
import keras

print("Keras Version: {keras.__version__}")

keras.config.set_image_data_format('channels_last')


# SET ALL THE PARAMETERS
weights_path = 'models/vgg16.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
img_width, img_height = 150, 150
train_data_dir = 'data/dogscats1000/train'
validation_data_dir = 'data/dogscats1000/validation'
nb_train_samples = 2000
nb_validation_samples = 2000
epochs = 30
batch_size = 16



# LOAD VGG16
input_tensor = Input(shape=(150, 150, 3))
model = VGG16(weights = 'imagenet', include_top = False, input_tensor = input_tensor)


# CREATE A TOP MODEL
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
#top_model.load_weights(top_model_weights_path)


# CREATE AN "REAL" MODEL FROM VGG16
# BY COPYING ALL THE LAYERS OF VGG16
new_model = Sequential()
for l in model.layers:
    new_model.add(l)


# CONCATENATE THE TWO MODELS
#new_model.add(top_model)

# LOCK THE TOP CONV LAYERS
for layer in new_model.layers:
    layer.trainable = False

# COMPILE THE MODEL
new_model.compile(loss='binary_crossentropy', optimizer = optimizers.SGD(learning_rate = 1e-4, momentum = 0.9),
                  metrics=['accuracy'])



# CREATE THE IMAGE GENERATORS
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size = (img_height, img_width),
                                                    batch_size = batch_size, class_mode = 'binary')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir, target_size = (img_height, img_width),
                                                              batch_size = batch_size, class_mode = 'binary')


#  FIT THE MODEL

new_model.fit_generator(train_generator, samples_per_epoch = nb_train_samples, epochs = epochs,
                        validation_data = validation_generator,
                        nb_val_samples = nb_validation_samples)

model.summary()